In [129]:
!pip install mtranslate

In [130]:
from bs4 import BeautifulSoup
import re
# import urllib2
import requests
import json

import mtranslate

In [131]:
url = "https://peoplepill.com/home-advanced-search-xhr"
#page = open(url)
#soup = BeautifulSoup(page.read())

#cities = soup.find_all('span', {'class' : 'city-sh'})

#for city in cities:
#    print city

urlCommonPart = "https://peoplepill.com/people/"
urlList = []


In [132]:
for i in range(1, 7):
  payload = {'currentPage':i,'namePart':'','countrySlug':'sri-lanka','occupationSlug':'cricketer','genderSlug':'','yearsRangeSlug':'1970-1999'}
  r = requests.post(url,data=payload)
  responseJSON = r.json()
  urlList = urlList + [(urlCommonPart + i['slug'] + '/') for i in responseJSON['paginator']['PeopleESMeta']]

In [133]:
urlList

['https://peoplepill.com/people/rumesh-buddika/',
 'https://peoplepill.com/people/shadeep-silva/',
 'https://peoplepill.com/people/mahela-jayawardene/',
 'https://peoplepill.com/people/ajantha-mendis/',
 'https://peoplepill.com/people/bathiya-perera/',
 'https://peoplepill.com/people/chirantha-de-silva/',
 'https://peoplepill.com/people/dilruk-laurence/',
 'https://peoplepill.com/people/mohamed-askar/',
 'https://peoplepill.com/people/nuwan-sameera/',
 'https://peoplepill.com/people/pasan-wanasinghe/',
 'https://peoplepill.com/people/ruwan-kalpage/',
 'https://peoplepill.com/people/saman-jayantha/',
 'https://peoplepill.com/people/shashikala-siriwardene/',
 'https://peoplepill.com/people/kaushal-silva/',
 'https://peoplepill.com/people/thisara-perera/',
 'https://peoplepill.com/people/pubudu-dassanayake/',
 'https://peoplepill.com/people/sujith-fernando/',
 'https://peoplepill.com/people/mario-villavarayan/',
 'https://peoplepill.com/people/vimukthi-perera/',
 'https://peoplepill.com/p

In [134]:
testPlayerURL = urlList[0]


In [135]:
# extract info from cricketer's page
def getCricketerInfo(url):
  req = requests.get(url)
  soup = BeautifulSoup(req.content, 'html.parser')
  res = {}

  res['name'] = soup.find('h1', class_='name').getText()
  res['info'] = soup.find('div', class_='brief-bio').getText()

  props = soup.select('tr.prop.border-bottom')
  for prop in props:
    label = prop.find(class_='label').getText().strip()
    #print(f'\t{label}')
    value = prop.find(class_='values').getText().strip()
    res[label] = value
  
  body = soup.find('div', 'body html-content')
  if body:
    texts = body.find_all(['p', 'h1', 'h2', 'h3'], recursive=False)
    text = ''
    for i in texts:
      text += i.get_text(separator='\n')
    res['description'] = text

  return res


In [136]:
allCricketersInfo = []

for cricketerURL in urlList:
  cricketerInfoDict = getCricketerInfo(cricketerURL)
  try:
    for x in cricketerInfoDict:
      translated = mtranslate.translate(cricketerInfoDict[x],"si","en")
      cricketerInfoDict[x] = translated
  except:
    continue

  #print(cricketerInfoDict)
  allCricketersInfo.append(cricketerInfoDict)


In [137]:
allCricketersInfo

[{'Age': 'අවුරුදු 31 යි',
  'Birth': '1990 නොවැම්බර් 14',
  'From': 'ශ්රී ලංකාව',
  'Gender': 'පිරිමි',
  'Intro': 'ශ්\u200dරී ලංකා ක්\u200dරිකට් ක්\u200dරීඩකයෙක්',
  'Is': 'මලල ක්රීඩකයා\nක්\u200dරිකට් ක්\u200dරීඩකයෙක්',
  'Type': 'ක්රීඩා',
  'description': 'රුමේෂ් බුද්ධික\n (උපත 1990 නොවැම්බර් 14) යනු ශ්\u200dරී ලංකා සංවර්ධන නැගී එන කණ්ඩායමේ නායකයා වන ශ්\u200dරී ලංකා ක්\u200dරිකට් ක්\u200dරීඩකයෙකි. ඔහු දැනට Burgher Recreation Club වෙනුවෙන් ක්\u200dරීඩා කරන වමත් පිතිකරුවෙක් සහ දකුණතින් ඕෆ් බ්\u200dරේක් පන්දු යවන්නෙක් වේ. ඔහු මාතර දිස්ත්\u200dරික්කයේ උපත ලබා ගාල්ල මහින්ද විද්\u200dයාලයෙන් අධ්\u200dයාපනය ලැබූ අතර එහිදී ඔහු ක්\u200dරිකට් දිවිය ආරම්භ කළේය.ඔහු 2010 නවසීලන්තයේ පැවති ICC වයස අවුරුදු 19 න් පහළ ක්\u200dරිකට් ලෝක කුසලානය සඳහා ශ්\u200dරී ලංකාව නියෝජනය කළේය. 2008 වසරේ හලාවත මේරියන්ස් කණ්ඩායම වෙනුවෙන් වයස 23 න් පහළ තරගාවලියට ක්\u200dරීඩා කළ බුද්ධික 2009 දී රාගම ක්\u200dරිකට් සමාජයට එක් වූ අතර ඔහු සමඟ එම තරගයේ අවසන් පූර්ව වටය දක්වා ඔවුන්ගේ වයස අවුරුදු 23 න් පහළ කණ්ඩායම සමඟින් තරග වැ

In [144]:
allCricketersInfoJSON = json.dumps(allCricketersInfo, ensure_ascii = False)

In [147]:
with open('data.json', 'w') as outfile:
    json.dump(allCricketersInfoJSON, outfile, ensure_ascii=False)